# Read from DB

In this case is a postgres, but any databse with support to jdbc driver should not be much different

this DOESNT really work yet, we have to download the driver and put it in the /drivers/ folder

and, of course, we don't have a database running

In [ ]:
spark = SparkSession.builder \
    .master('local') \
    .appName('myAppName') \
    .config('spark.executor.memory', '4gb') \
    .config('spark.cores.max', '4') \
    .config('spark.sql.legacy.setCommandRejectsSparkCoreConfs', 'false') \
    .config('spark.driver.extraClassPath', '../drivers/*') \
    .getOrCreate()
    
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark.conf.set("spark.driver.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true")
spark.conf.set("spark.executor.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true")
spark.conf.set('spark.executorEnv.ARROW_PRE_0_15_IPC_FORMAT', "1")

df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgres://...") \
    .option("user", "...") \
    .option("password", "...") \
    .option("query", "SELECT * FROM LALALA")\
    .load()
print(df.count())

# Read from S3

Again, I'm not sure if this run as-it-is without installing anything else

But I did tested this code in AWS EMR, where it doesnt need anything else

also, probably it's possible to read without specifying the schema

In [ ]:
schema = StructType([
    StructField('created_at', TimestampType(), True),
    StructField('value', StringType(), True),
])


df = spark.read\
    .schema(schema)\
    .options(header='true', multiLine=True, delimiter = ",", mode="PERMISSIVE")\
    .csv("s3://crazy-bucket/lalala.csv")

print(df.count())
df.printSchema()
df.show(10)
